In [6]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [200]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\non_linear\non_linear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\non_linear\model_fft')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear
Folder successfully created at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\non_linear\model_fft


In [201]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\non_linear\non_linear.pkl


In [202]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [203]:
u = torch.Tensor(dataset['y_train'])

f_coef = torch.fft.fft2(u)
F_ordered = f_coef.flatten(start_dim=1)

mask = np.zeros_like(F_ordered)
mask[:, 0:3] = 1

F_full = f_coef*mask.reshape(f_coef.shape)
u_reconstructed = torch.fft.ifft2(F_full).real

# y_train = F_ordered[:, 0:3].reshape(u.shape[0], 3)

real_part = F_ordered[:, 0:3].reshape(u.shape[0], 3).real
imag_part = F_ordered[:, 0:3].reshape(u.shape[0], 3).imag

y_train = torch.stack([
    real_part,
    imag_part,
], dim=-1)

In [204]:
y_train.shape

torch.Size([8000, 3, 2])

In [208]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True)
f_train = torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True)

In [209]:
input_size = X_train[0].shape
predictive_output_size = y_train[0].shape
explanatory_output_size = K_train[0].shape

In [248]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class FFTNeuralNetwork(nn.Module):

    def __init__(self, input_size, predictive_output_size, explanatory_output_size, **kwargs):
        super(FFTNeuralNetwork, self).__init__()

        self.input = input_size
        self.output_pred = predictive_output_size
        self.output_expl = explanatory_output_size

        self.hidden_units_pred = 10
        self.hidden_units_exp = 15
        self.filters_exp = 10

        # Predictive network
        self.flatten_layer_pred = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer_pred = nn.Linear(torch.prod(torch.tensor(self.input)), self.hidden_units_pred)  
        self.hidden2_layer_pred = nn.Linear(self.hidden_units_pred, self.hidden_units_pred)  
        self.output_layer_pred = nn.Linear(self.hidden_units_pred, self.output_pred[0]*self.output_pred[1])

        # Explanatory network


    def forward(self, X):

        # Predictive network
        X = self.flatten_layer_pred(X)
        X = torch.sigmoid(self.hidden1_layer_pred(X))
        X = torch.sigmoid(self.hidden2_layer_pred(X))
        output_predictive_net = self.output_layer_pred(X)

        # Obtain real and imaginary part of output and transform it in a complex number
        output_predictive = output_predictive_net.view(output_predictive_net.size(0), self.output_pred[0], self.output_pred[1])
        real = output_predictive[..., 0]
        imag = output_predictive[..., 1]
        
        # Reconstruction of u(x, y) with iFFT
        f_coef = torch.zeros(output_predictive_net.size(0), 1, self.output_expl[0], self.output_expl[1], dtype=torch.complex64)
        f_coef[:, 0, 0:3, 0] = torch.complex(real, imag)
        u_reconstructed = TensOps(torch.fft.ifft2(f_coef).real, space_dimension=2, contravariance=0, covariance=0)


        


model = FFTNeuralNetwork(input_size, predictive_output_size, explanatory_output_size)
model(X_train)

In [ ]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class FFTNeuralNetwork(nn.Module):

    def __init__(self, input_size, hidden1_dim, hidden2_dim, output_size, n_filters=5, **kwargs):
        super(FFTNeuralNetwork, self).__init__()

        self.input_size = input_size
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim
        self.output_size = output_size
        self.n_filters = n_filters

        # Predictive network
        self.flatten_layer_pred = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer_pred = nn.Linear(torch.prod(torch.tensor(self.input_size)), self.hidden1_dim)  
        self.hidden2_layer_pred = nn.Linear(self.hidden1_dim, self.hidden2_dim)  
        self.output_layer_pred = nn.Linear(self.hidden2_dim, self.output_size[2] * self.output_size[2])  

        # Explanatory network
        self.conv1_exp = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=1)
        self.flatten_layer_exp = nn.Flatten()
        self.hidden1_layer_exp = nn.LazyLinear(hidden1_dim)
        self.hidden2_layer_exp = nn.Linear(hidden1_dim, hidden2_dim)
        self.output_layer_exp = nn.Linear(hidden2_dim, n_filters * (output_size[1] - 1) * (output_size[2] - 1))
        self.conv2_exp = nn.Conv2d(in_channels=n_filters, out_channels=1, kernel_size=1)
    
    def forward(self, X):

        # Predictive network
        X_pred_flat = self.flatten_layer_pred(X)
        X_pred_hidden1 = torch.sigmoid(self.hidden1_layer_pred(X_pred_flat))
        X_pred_hidden2 = torch.sigmoid(self.hidden2_layer_pred(X_pred_hidden1))
        output_dense_pred = self.output_layer_pred(X_pred_hidden2)

        # Mean operator
        u_pred = output_dense_pred.view(output_dense_pred.size(0), 1, self.output_size[1], self.output_size[2])
        um_pred = Mx(My(TensOps(u_pred, space_dimension=2, contravariance=0, covariance=0))).values

        
        # Explanatory network
        x = torch.sigmoid(self.conv1_exp(um_pred))
        x = self.flatten_layer_exp(x)
        x = torch.sigmoid(self.hidden1_layer_exp(x))
        x = torch.sigmoid(self.hidden2_layer_exp(x))
        x = self.output_layer_exp(x)
        x = x.view(x.size(0), self.n_filters, self.output_size[1] - 1, self.output_size[2] - 1)
        K_pred = self.conv2_exp(x)

        return u_pred, K_pred

